In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import re 

In [2]:
# Keras 2.2.4/tensorflow 1.14.0 -> Version matching for local machine

from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras import initializers


In [3]:
myname = 'Gagandeep'

#Using os.walk to walk through each file in the directory provided
files = []
import os
for dirname, _, filenames in os.walk('gagan_Oct_24_2019_data/'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        files.append(os.path.join(dirname, filename))

file_list = []
unique_file_list = []    


# For every file in the hierarchy this regex operation takes files only with N=1 or N=28

for value in files:
    x = re.search(".*28_poly_perf.csv$",value)
#     x = re.search(a,value)
    if(x):
#         print(x.group(0))
        file_list.append(x.group(0))
    y = re.search("\w+/\w+/",value)
    if(y):
        if y.group(0) not in unique_file_list:
            unique_file_list.append(y.group(0))

#Sorts the ordering of files
file_list.sort()


In [4]:

# In[3]:


# For particular file we are generating various permutations 
arr = []
for diff_file in file_list:
# for layer in range(1,20):
    train_df = pd.read_csv(diff_file,header=None)
    train_df = train_df.drop([0,2,3,4,5,6,7,8,9], axis=1)

    train_df = train_df.sort_values(by=[1],ascending=False)

    train_df = train_df.dropna(axis=1)
    npdata = np.array(train_df)
    for i in range(len(npdata)):
        for j in range(len(npdata)):
            arr.append(np.concatenate((npdata[i],npdata[j]), axis=0))
arr = np.asarray(arr)

ver_2_GF = npdata.shape[1]


# For each data point we are creating the output as 01 or 10 and concatenating with the file
out1 = []
out2 = []

for i in range(len(arr)):
    if arr[i][0] >= arr[i][ver_2_GF]:
        out1.append(1)
        out2.append(0)

    else:
        out1.append(0)
        out2.append(1)

arr = pd.DataFrame(arr)
arr = arr.drop([0,ver_2_GF],axis=1)
arr["sum"] = arr.sum(axis=1)
arr = arr.loc[:,[1,2,3,4,6,7,8,9]].div(arr["sum"], axis=0)
arr.head()

out1 = pd.DataFrame(out1)
out2 = pd.DataFrame(out2)

arr = pd.concat([arr, out1, out2,], axis=1, sort=False)
arr = np.array(arr)
print(arr.shape)



(45052, 10)


In [9]:

# In[4]:


# Till Here I have preprocessed the data properly.
# Splitting the data into 70% training and 30% test data.
length = int(0.70*(len(arr)))
training_set = arr[:length]
test_set = arr[length:]


# In[5]:


#Initializing the keras Model
model = Sequential()
initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=2)
model.add(Dense(32, input_dim=8, activation='relu',kernel_initializer='random_uniform'))
model.add(Dense(20, activation='relu',kernel_initializer='random_uniform'))
model.add(Dense(16, activation='relu',kernel_initializer='random_uniform'))
model.add(Dense(12, activation='softsign',kernel_initializer='random_uniform'))
model.add(Dense(8, activation='relu',kernel_initializer='random_uniform'))
model.add(Dense(2, activation='softmax',kernel_initializer='random_uniform'))


# compile the keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(training_set[:,:-2],training_set[:,8:], epochs=8, batch_size=8)
# evaluate the keras model
_, accuracy = model.evaluate(test_set[:,:-2],test_set[:,8:])
print('Accuracy: %.2f' % (accuracy*100))



Epoch 1/8
3942/3942 [==============================] - 5s 1ms/step - loss: 0.5973 - accuracy: 0.6591
Epoch 2/8
3942/3942 [==============================] - 4s 1ms/step - loss: 0.5680 - accuracy: 0.6917
Epoch 3/8
3942/3942 [==============================] - 4s 1ms/step - loss: 0.5603 - accuracy: 0.6931
Epoch 4/8
3942/3942 [==============================] - 5s 1ms/step - loss: 0.5553 - accuracy: 0.6980
Epoch 5/8
3942/3942 [==============================] - 5s 1ms/step - loss: 0.5549 - accuracy: 0.6976
Epoch 6/8
3942/3942 [==============================] - 5s 1ms/step - loss: 0.5519 - accuracy: 0.6993
Epoch 7/8
3942/3942 [==============================] - 5s 1ms/step - loss: 0.5494 - accuracy: 0.7035
Epoch 8/8
423/423 [==============================] - 0s 868us/step - loss: 0.5945 - accuracy: 0.6833
Accuracy: 68.33


In [8]:

# In[6]:


# Again for a file we are making all permutations and are passing through Model for prediction.
for diff_file in file_list:
# for layer in range(1,20):
    arr_predict = []
    train_df = pd.read_csv(diff_file,header=None)
    
    myname = "results_"+diff_file
    train_df = train_df.sort_values(by=[1],ascending=False)
    print_ver_GF = train_df.dropna(axis=1)
    print_ver_GF = np.asarray(print_ver_GF)
#     print(print_ver_GF[0,1])
    train_df = train_df.drop([0,1,2,3,4,5,6,7,8,9], axis=1)
    train_df = train_df.dropna(axis=1)
    npdata = np.array(train_df)
    for i in range(len(npdata)):
        for j in range(len(npdata)):
            arr_predict.append(np.concatenate((npdata[i],npdata[j]), axis=0))
    arr_predict = np.asarray(arr_predict)
    prediction = model.predict(arr_predict)

# Uptill here we have the predicted output data ready.
# Now we calculate number of wins for each data point and append (ActualRank, GFlops, Version, wins,
# PolyRank) values in "print_result" list, for each file

    version =0
    flag=0
    count_win =0
    distinct_wins = []
    Actual_rank = 0
    print_result = []
    for x in prediction:
        print(x)
    
    for values in prediction[:,0]:
        if flag<len(npdata):
            flag +=1
            if values>0.5:
                count_win+=1
        else:
            print_result.append([Actual_rank+1,print_ver_GF[Actual_rank,1],print_ver_GF[Actual_rank,0],count_win])
            Actual_rank+=1
            version+=1
            if count_win not in distinct_wins:
                distinct_wins.append(count_win)
            count_win=0
            flag=1
            if values>0.5:
                count_win+=1
    print_result.append([Actual_rank+1,print_ver_GF[Actual_rank,1],print_ver_GF[Actual_rank,0],count_win])
    if count_win not in distinct_wins:
        distinct_wins.append(count_win)
    
# On the basis of Number of wins, We are calculating the number of Polyrank for each data point.
    distinct_wins.sort(reverse = True)
    polyrank = []
    for values in print_result:
        polyrank.append(distinct_wins.index(values[3])+1)
    polyrank = pd.DataFrame(polyrank)
    Final = pd.concat([pd.DataFrame(print_result),polyrank], axis=1, sort=False)
    Final= np.asarray(Final)

    temp = Final[Final[:,4] == 1]

    num_row_in_top5_per = math.ceil((Final.shape[0])*0.05)
    rank = 1

# Calculating best in Top 5%
    top5_max_list =[]    
    while num_row_in_top5_per>0:
        temp_rank = Final[Final[:,4] == rank]
        top5_max_list.append(np.max(temp[:,1]))
        num_row_in_top5_per-=temp_rank.shape[0]
        rank+=1
        
    top5_perf_list = [np.max(Final[:,1]),np.max(temp[:,1]),Final.shape[0],max(top5_max_list),np.min(Final[:,1]),np.median(Final[:,1])]
    print("Hello",top5_perf_list)

# Writing the Wins, polyrank etc to a file.
    f = open(myname, "a+") 
    f.write('ActualRank, GFlops, Version, wins, PolyRank,\n')
    f.close()
    for values in Final:
        f = open(myname, "a+")        
        str_temp = ""
        for val in values:
            str_temp = str_temp + str(val)+", "
        f.write("%s\n" % str_temp)
        f.close()

# Creating summary files  and writing it to a file
    myname_summary = "summary_results_28"
    string2 = re.search("\w+/\w+/",diff_file).group(0)
    if string2 in diff_file:
        string2 = diff_file.replace(string2,'')
    if "_poly_perf.csv" in string2:
        string2 = string2.replace("_poly_perf.csv",'')
        
    f = open(myname_summary, "a")
#     f.write("Max_GFLOPS, Poly_Top_1GFLOPS,numVariants,Poly_Top_0.050000,Min_GFLOPS,Median_GFLOPS,\n")
    str_temp = string2+", "
    
    for val in top5_perf_list:
        str_temp = str_temp + str(val) + ", "
    f.write("%s\n" % str_temp)
    f.close()

[0.5252883 0.4747117]
[0.5252883 0.4747117]
[0.5249832 0.4750168]
[0.52484566 0.47515437]
[0.5249668  0.47503328]
[0.5249536 0.4750464]
[0.5249536 0.4750464]
[0.5241157  0.47588426]
[0.5249536 0.4750464]
[0.5249277 0.4750723]
[0.5252883 0.4747117]
[0.5252883 0.4747117]
[0.5249832 0.4750168]
[0.52484566 0.47515437]
[0.5249668  0.47503328]
[0.5249536 0.4750464]
[0.5249536 0.4750464]
[0.5241157  0.47588426]
[0.5249536 0.4750464]
[0.5249277 0.4750723]
[0.5249714  0.47502854]
[0.5249714  0.47502854]
[0.5249568  0.47504327]
[0.5245385 0.4754615]
[0.5249454  0.47505462]
[0.5249224  0.47507766]
[0.5249224  0.47507766]
[0.52428824 0.47571167]
[0.5249224  0.47507766]
[0.5248582  0.47514185]
[0.5249595  0.47504047]
[0.5249595  0.47504047]
[0.5249564  0.47504362]
[0.5249374  0.47506258]
[0.5249543  0.47504565]
[0.52494574 0.4750543 ]
[0.52494574 0.4750543 ]
[0.5249482  0.47505185]
[0.52494574 0.4750543 ]
[0.52493936 0.47506055]
[0.52496874 0.4750312 ]
[0.52496874 0.4750312 ]
[0.52495086 0.47504908

[0.52493083 0.4750691 ]
[0.52493083 0.4750691 ]
[0.52493083 0.4750691 ]
[0.52494    0.47505996]
[0.52411413 0.4758858 ]
[0.52411413 0.4758858 ]
[0.523338   0.47666198]
[0.5240548  0.47594523]
[0.52493083 0.4750691 ]
[0.52493083 0.4750691 ]
[0.52493083 0.4750691 ]
[0.52494    0.47505996]
[0.52411413 0.4758858 ]
[0.52411413 0.4758858 ]
[0.523338   0.47666198]
[0.5240548  0.47594523]
[0.52493083 0.4750691 ]
[0.52493083 0.4750691 ]
[0.52493083 0.4750691 ]
[0.52494    0.47505996]
[0.52411413 0.4758858 ]
[0.52411413 0.4758858 ]
[0.523338   0.47666198]
[0.5240548  0.47594523]
[0.52493143 0.47506857]
[0.52493143 0.47506857]
[0.52493143 0.47506857]
[0.52494055 0.47505942]
[0.5241051  0.47589487]
[0.5241051  0.47589487]
[0.5233368 0.4766631]
[0.5240151  0.47598484]
[0.5233368 0.4766631]
[0.5233368 0.4766631]
[0.5233368 0.4766631]
[0.5233371  0.47666293]
[0.5241376  0.47586232]
[0.5241376  0.47586232]
[0.52412665 0.4758734 ]
[0.52413976 0.47586027]
[0.5233368 0.4766631]
[0.5233368 0.4766631]
[0.5

[0.524177   0.47582304]
[0.5241061  0.47589394]
[0.5241061  0.47589394]
[0.5241974  0.47580257]
[0.5240185  0.47598144]
[0.5240185  0.47598144]
[0.5240185  0.47598144]
[0.5233572 0.4766428]
[0.52421165 0.47578835]
[0.5240858  0.47591415]
[0.5240858  0.47591415]
[0.52422595 0.475774  ]
[0.5233379 0.4766621]
[0.5233379 0.4766621]
[0.5233379 0.4766621]
[0.5233348 0.4766652]
[0.52421165 0.47578835]
[0.5240858  0.47591415]
[0.5240858  0.47591415]
[0.52422595 0.475774  ]
[0.5233379 0.4766621]
[0.5233379 0.4766621]
[0.5233379 0.4766621]
[0.5233348 0.4766652]
[0.52416533 0.47583464]
[0.52400565 0.4759944 ]
[0.52400565 0.4759944 ]
[0.52418256 0.47581744]
[0.52334267 0.4766573 ]
[0.52334267 0.4766573 ]
[0.52334267 0.4766573 ]
[0.523335   0.47666502]
[0.52332014 0.47667992]
[0.5233202  0.47667983]
[0.5233202  0.47667983]
[0.52331996 0.47667998]
[0.5240753  0.47592467]
[0.5240753  0.47592467]
[0.5240753  0.47592467]
[0.5240928 0.4759072]
[0.52332014 0.47667992]
[0.5233202  0.47667983]
[0.5233202  

Hello [1181.8, 1181.8, 14, 1181.8, 1060.3, 1082.85]
[0.52412343 0.47587654]
[0.52412313 0.47587687]
[0.52412033 0.47587967]
[0.52412033 0.47587967]
[0.52412033 0.47587967]
[0.52412343 0.47587654]
[0.52412313 0.47587687]
[0.52412343 0.47587654]
[0.52412033 0.47587967]
[0.52412343 0.47587654]
[0.52412033 0.47587967]
[0.52412033 0.47587967]
[0.52334213 0.4766578 ]
[0.52334213 0.4766578 ]
[0.52411956 0.4758804 ]
[0.52411884 0.47588113]
[0.5241182  0.47588181]
[0.5241182  0.47588181]
[0.5241182  0.47588181]
[0.52411956 0.4758804 ]
[0.52411884 0.47588113]
[0.52411956 0.4758804 ]
[0.5241182  0.47588181]
[0.52411956 0.4758804 ]
[0.5241182  0.47588181]
[0.5241182  0.47588181]
[0.5233235  0.47667652]
[0.5233235  0.47667652]
[0.52413946 0.47586048]
[0.52414024 0.47585973]
[0.5241324  0.47586754]
[0.5241324  0.47586754]
[0.5241324  0.47586754]
[0.52413946 0.47586048]
[0.52414024 0.47585973]
[0.52413946 0.47586048]
[0.5241324  0.47586754]
[0.52413946 0.47586048]
[0.5241324  0.47586754]
[0.5241324  

[0.524096   0.47590405]
[0.524096   0.47590405]
[0.524096   0.47590405]
[0.524096   0.47590405]
[0.5240841  0.47591597]
[0.52408487 0.4759151 ]
[0.5240841  0.47591597]
[0.5240849 0.4759151]
[0.5240842 0.4759158]
[0.5240842 0.4759158]
[0.5240841  0.47591597]
[0.5240841  0.47591597]
[0.5240884  0.47591168]
[0.5240914  0.47590855]
[0.52409375 0.47590622]
[0.52409375 0.47590622]
[0.52409375 0.47590622]
[0.52409375 0.47590622]
[0.52408695 0.47591305]
[0.5240876  0.47591233]
[0.52408695 0.47591305]
[0.52408767 0.47591236]
[0.5240871  0.47591296]
[0.5240871  0.47591296]
[0.52408695 0.47591305]
[0.52408695 0.47591305]
[0.5240901  0.47590986]
[0.5240933  0.47590667]
[0.5240961 0.475904 ]
[0.5240961 0.475904 ]
[0.5240961 0.475904 ]
[0.5240961 0.475904 ]
[0.52408385 0.4759161 ]
[0.52408475 0.4759153 ]
[0.52408385 0.4759161 ]
[0.52408475 0.47591525]
[0.5240841 0.475916 ]
[0.5240841 0.475916 ]
[0.52408385 0.4759161 ]
[0.52408385 0.4759161 ]
[0.5240882 0.4759118]
[0.5240913 0.4759087]
[0.5240937 0.4

...

[0.52409756 0.47590238]
[0.5240954  0.47590455]
[0.52411723 0.47588286]
[0.52331793 0.47668207]
[0.52331793 0.47668207]
[0.52408653 0.47591344]
[0.5240863 0.4759137]
[0.52332044 0.47667956]
[0.52407664 0.47592336]
[0.52407664 0.47592336]
[0.5240957  0.47590426]
[0.5240957  0.47590426]
[0.52407664 0.47592336]
[0.5240871  0.47591296]
[0.52331996 0.47667998]
[0.52331805 0.476682  ]
[0.52331805 0.476682  ]
[0.5240892  0.47591072]
[0.52408904 0.47591093]
[0.52332175 0.47667828]
[0.52408373 0.47591633]
[0.52408373 0.47591633]
[0.52409655 0.47590336]
[0.52409655 0.47590336]
[0.52408373 0.47591633]
[0.52408963 0.47591037]
[0.5233209  0.47667906]
[0.5233221  0.47667795]
[0.5233221  0.47667795]
[0.52411133 0.47588873]
[0.524118   0.47588202]
[0.52409446 0.47590554]
[0.5241044 0.4758956]
[0.5241044 0.4758956]
[0.5240938  0.47590613]
[0.5240938  0.47590613]
[0.5241044 0.4758956]
[0.524104   0.47589606]
[0.5240918 0.4759082]
[0.52331835 0.47668168]
[0.52331835 0.47668168]
[0.52408993 0.47590998]
[0

[0.5240968  0.47590318]
[0.52410024 0.4758997 ]
[0.5240859  0.47591406]
[0.5240859  0.47591406]
[0.52410024 0.4758997 ]
[0.52410024 0.4758997 ]
[0.52410024 0.4758997 ]
[0.52407724 0.47592273]
Hello [2603.1, 2603.1, 12, 2603.1, 1800.1, 1973.9]
[0.52408886 0.4759111 ]
[0.52408886 0.4759111 ]
[0.52408886 0.4759111 ]
[0.52408886 0.4759111 ]
[0.52408886 0.4759111 ]
[0.5241131  0.47588682]
[0.5241131  0.47588682]
[0.5241131  0.47588682]
[0.52408886 0.4759111 ]
[0.52408886 0.4759111 ]
[0.52408886 0.4759111 ]
[0.52408886 0.4759111 ]
[0.52408886 0.4759111 ]
[0.5241131  0.47588682]
[0.5241131  0.47588682]
[0.5241131  0.47588682]
[0.52408886 0.4759111 ]
[0.52408886 0.4759111 ]
[0.52408886 0.4759111 ]
[0.52408886 0.4759111 ]
[0.52408886 0.4759111 ]
[0.5241131  0.47588682]
[0.5241131  0.47588682]
[0.5241131  0.47588682]
[0.52408886 0.4759111 ]
[0.52408886 0.4759111 ]
[0.52408886 0.4759111 ]
[0.52408886 0.4759111 ]
[0.52408886 0.4759111 ]
[0.5241131  0.47588682]
[0.5241131  0.47588682]
[0.5241131  0

[0.52411294 0.47588712]
[0.52411294 0.47588712]
[0.52411294 0.47588712]
[0.5233296 0.4766704]
[0.5233308  0.47666922]
[0.5233308  0.47666922]
[0.5233324  0.47666758]
[0.5233308  0.47666922]
[0.5233308  0.47666922]
[0.5233286  0.47667137]
[0.52334577 0.47665417]
[0.52410865 0.47589132]
[0.5241089  0.47589108]
[0.5241089  0.47589108]
[0.5241089  0.47589108]
[0.5233249  0.47667506]
[0.52332526 0.4766747 ]
[0.52332526 0.4766747 ]
[0.52332574 0.4766743 ]
[0.52332526 0.4766747 ]
[0.52332526 0.4766747 ]
[0.52332467 0.47667533]
[0.5233348 0.4766652]
[0.52408874 0.4759113 ]
[0.5240996  0.47590044]
[0.5240996  0.47590044]
[0.5240996  0.47590044]
[0.5233225  0.47667742]
[0.52332294 0.47667706]
[0.52332294 0.47667706]
[0.5233231  0.47667694]
[0.52332294 0.47667706]
[0.52332294 0.47667706]
[0.52332246 0.47667754]
[0.52332413 0.47667593]
[0.523986 0.476014]
[0.52407783 0.47592217]
[0.52407783 0.47592217]
[0.52407783 0.47592217]
[0.5233225  0.47667742]
[0.52332294 0.47667706]
[0.52332294 0.47667706]


[0.5241113  0.47588876]
[0.524105   0.47589493]
[0.52405787 0.4759422 ]
[0.5240576  0.47594243]
[0.5240539  0.47594616]
[0.5240576  0.47594243]
[0.5240539  0.47594616]
[0.52406317 0.4759368 ]
[0.5240643  0.47593567]
[0.5240539  0.47594616]
[0.5240576  0.47594243]
[0.5240539  0.47594616]
[0.5240615  0.47593853]
[0.523888   0.47611195]
[0.5241041  0.47589594]
[0.524102   0.47589806]
[0.5241041  0.47589594]
[0.5241041  0.47589594]
[0.5241041  0.47589594]
[0.5240955 0.4759045]
[0.5233216 0.4766783]
[0.52332187 0.47667807]
[0.52332187 0.4766781 ]
[0.52332187 0.47667807]
[0.52332187 0.4766781 ]
[0.523322   0.47667795]
[0.523322   0.47667792]
[0.52332187 0.4766781 ]
[0.52332187 0.47667807]
[0.52332187 0.4766781 ]
[0.52332175 0.47667828]
[0.523321   0.47667903]
[0.5240803  0.47591975]
[0.5233318  0.47666815]
[0.5240803  0.47591975]
[0.5240803  0.47591975]
[0.5240803  0.47591975]
[0.52332383 0.4766761 ]
[0.5233317 0.4766684]
[0.5233318  0.47666818]
[0.52333146 0.47666857]
[0.5233318  0.47666818

Hello [2480.6, 2480.6, 16, 2480.6, 1236.8, 2457.6]
[0.52409947 0.47590053]
[0.52409947 0.47590053]
[0.5241198  0.47588015]
[0.524109 0.475891]
[0.52411306 0.47588697]
[0.5241128 0.4758872]
[0.5241149  0.47588506]
[0.5241149  0.47588506]
[0.52409434 0.47590572]
[0.52409434 0.47590572]
[0.5241149  0.47588506]
[0.5240936 0.4759064]
[0.52409947 0.47590053]
[0.52409947 0.47590053]
[0.5241198  0.47588015]
[0.524109 0.475891]
[0.52411306 0.47588697]
[0.5241128 0.4758872]
[0.5241149  0.47588506]
[0.5241149  0.47588506]
[0.52409434 0.47590572]
[0.52409434 0.47590572]
[0.5241149  0.47588506]
[0.5240936 0.4759064]
[0.5233233  0.47667667]
[0.5233233  0.47667667]
[0.52409655 0.47590345]
[0.52411026 0.4758898 ]
[0.5240982  0.47590178]
[0.5240968  0.47590327]
[0.5240976  0.47590235]
[0.5240976  0.47590235]
[0.52410644 0.4758936 ]
[0.52410644 0.4758936 ]
[0.5240976  0.47590235]
[0.5241061  0.47589388]
[0.523322   0.47667792]
[0.523322   0.47667792]
[0.5240868  0.47591332]
[0.5240972  0.47590274]
[0.52